# ChatGPT Speech Assistant Playbook

<p>
Mal Minhas, v0.1<br>
16.03.23
</p>

### Introduction

The recipe here was taken from the work of Faizan Bashir written up in a nice blog post [here](https://faizanbashir.me/building-a-chatgpt-based-ai-assistant-with-python-speech-to-text-and-text-to-speech-using-openai-apis).  The recipe outlined below is basically a copy of his work.  There are FOUR separate stages in this order:
1. `speech_recognition` is used to record audio input to a WAV file using `portaudio`
2. OpenAI [Whisper API](https://openai.com/research/whisper) is used to convert the WAV file to a text prompt as speech to text (STT). 
3. OpenAI `gpt-3.5-turbo` model is used to process the text prompt and generate a response
4. [`pyttsx3`](https://pyttsx3.readthedocs.io/en/latest/engine.html) is used to vocalise the ChatGPT response as text to speech (TTS).

### Installation

Following Faizan's instructions for a MacBook:
* Do a `brew install portaudio`
* Create a virtualenv let's say `chatgpt`
* `pip install SpeechRecognition, pyttsx3, requests` into `chatgpt`
* Ensure you have a valid OpenAI API token in an `OPENAI_API_TOKEN` environment variable

Known Issues:
* There is typically a 10 second or so gap between the utterance and the ChatGPT response
* `pyttsx3` is blocking and once you start an utterance, you don't seem to be able to stop it from completing even if you interrupt the kernel

### Code

In [1]:
import os
import speech_recognition as sr
import requests
import pyttsx3

AUDIO_FOLDER = "./audio"
INPUT_FILENAME = "microphone-results"
OPENAI_URL = "https://api.openai.com/v1"
OPENAI_TOKEN = os.environ.get("OPENAI_API_TOKEN")
#OPENAI_MODEL = 'gpt-3.5-turbo'
OPENAI_MODEL = 'gpt-4'
assert(OPENAI_TOKEN)

def recordSpeech():
    ''' Obtain audio fr|om microphone using python speech_recognition and return speech WAV file. '''
    print('[1.] Record audio using microphone')
    # obtain audio from the microphone
    r = sr.Recognizer()
    with sr.Microphone() as source:
        r.adjust_for_ambient_noise(source)
        print("Say something then a gap to finish!")
        audio = r.listen(source)
    audio_file_path = f"{AUDIO_FOLDER}/{INPUT_FILENAME}.wav"
    if not os.path.exists(AUDIO_FOLDER):
        os.mkdir(AUDIO_FOLDER)
    # write audio to a WAV file
    #print(f"Generating WAV file, saving at location: {audio_file_path}")
    with open(audio_file_path, "wb") as f:
        f.write(audio.get_wav_data())
    return audio_file_path

def convertSpeechToText(audio_file_path):
    ''' Convert speech WAV file to text. '''
    print('[2.] Call to Whisper API\'s to get the STT response')
    url = f'{OPENAI_URL}/audio/transcriptions'
    data = {
        'model': 'whisper-1',
        'file': audio_file_path,
    }
    files = {
        'file': open(audio_file_path, "rb")
    }
    headers = { 'Authorization' : f'Bearer {OPENAI_TOKEN}' }
    response = requests.post(url, files=files, data=data, headers=headers)
    #print("Status Code", response.status_code)
    speech_to_text = response.json()['text']
    #print("Response from Whisper API's", speech_to_text)
    return speech_to_text

def getChatGPTresponse(prompt_text):
    ''' '''
    print(f'[3.] Querying ChatGPT model with the STT response data for prompt:\n"{prompt_text}"')
    url = f'{OPENAI_URL}/chat/completions'
    data = {
        'model': OPENAI_MODEL,
        'messages': [
            {
                'role': 'user',
                'content': prompt_text
            }
        ]
    }
    headers = { 'Authorization' : f'Bearer {OPENAI_TOKEN}' }
    response = requests.post(url, json=data, headers=headers)
    #print("Status Code", response.status_code)
    chatgpt_response = response.json()['choices'][0]['message']['content'].strip()
    print(f'Response from ChatGPT \'{OPENAI_MODEL}\' model:\n"{chatgpt_response}"')
    return chatgpt_response

def convertTextToSpeech(chatgpt_response, speed=200):
    engine = pyttsx3.init()
    print(f'[4.] Try to convert TTS from the response, speed={speed}')
    def onStart(name):
        print(f'started-utterance')
    def onWord(name, location, length):
        pass
    def onError(name, location, length):
        print(f'error: {name}')
        engine.stop()
    def onEnd(name, completed):
        print(f'finished-utterance')
    started = engine.connect('started-utterance', onStart)
    error = engine.connect('error', onError)
    #engine.connect('started-word', onWord)
    finished = engine.connect('finished-utterance', onEnd)
    engine.setProperty('rate', speed)
    #print("Converting text to speech...")
    engine.say(chatgpt_response)
    try:
        engine.runAndWait()
    except Exception as e:
        print(f'stopping because "{e}"')
    engine.disconnect(started)
    engine.disconnect(error)
    engine.disconnect(finished)
    engine.stop()

In [2]:
speech = recordSpeech()
prompt_text = convertSpeechToText(speech)
chatgpt_response = getChatGPTresponse(prompt_text)
convertTextToSpeech(chatgpt_response)

[1.] Record audio using microphone
Say something then a gap to finish!
[2.] Call to Whisper API's to get the STT response
[3.] Querying ChatGPT model with the STT response data for prompt:
"Why is the sky blue?"
Response from ChatGPT 'gpt-4' model:
"The sky appears blue because of a phenomenon called Rayleigh scattering. When sunlight enters the Earth's atmosphere, it is made up of a spectrum of colors. The gas molecules and other small particles in the atmosphere scatter the sunlight in all directions. 

Blue light has a shorter wavelength and is scattered more easily than other colors with longer wavelengths, like red and yellow. As a result, when we look at the sky, the blue light is scattered across the atmosphere and dominates our field of vision, making the sky appear blue."
[4.] Try to convert TTS from the response, speed=200
started-utterance
finished-utterance
